In [ ]:
# ================= Step 1: Install YOLOv5 =================
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
%pip install -qr requirements.txt  # install dependencies

import torch
from IPython.display import Image, display
import os
print("YOLOv5 version:", torch.__version__)


In [ ]:
import os
import shutil
from sklearn.model_selection import train_test_split
import glob

# Paths
images_path = '/content/data/images' 
labels_path = '/content/data/labels'

# Create YOLOv5-style folders
os.makedirs('/content/data/images/train', exist_ok=True)
os.makedirs('/content/data/images/val', exist_ok=True)
os.makedirs('/content/data/labels/train', exist_ok=True)
os.makedirs('/content/data/labels/val', exist_ok=True)

# Get all image paths
image_files = glob.glob(os.path.join(images_path, '*.jpg'))

# Split into train and val
train_imgs, val_imgs = train_test_split(image_files, test_size=0.2, random_state=42)

# Helper function to move image and label pairs
def move_to_subfolder(image_list, target_img_dir, target_label_dir):
    for img in image_list:
        filename = os.path.basename(img)
        labelname = filename.replace('.jpg', '.txt')
        label_path = os.path.join(labels_path, labelname)

        shutil.copy(img, os.path.join(target_img_dir, filename))
        if os.path.exists(label_path):
            shutil.copy(label_path, os.path.join(target_label_dir, labelname))

# Move files
move_to_subfolder(train_imgs, '/content/data/images/train', '/content/data/labels/train')
move_to_subfolder(val_imgs, '/content/data/images/val', '/content/data/labels/val')


In [ ]:
yaml_text = """
train: /content/data/images/train
val: /content/data/images/val

nc: 1
names: ['tumor']
"""

with open('/content/data/tumor_dataset.yaml', 'w') as f:
    f.write(yaml_text)

print("tumor_dataset.yaml created!")


In [ ]:
import os

label_dirs = ['/content/data/labels/train', '/content/data/labels/val']

for label_dir in label_dirs:
    for fname in os.listdir(label_dir):
        if fname.endswith('.txt'):
            fpath = os.path.join(label_dir, fname)
            with open(fpath, 'r') as file:
                lines = file.readlines()

            # Keep only class 1 (tumor), change it to 0
            new_lines = []
            for line in lines:
                parts = line.strip().split()
                if parts and parts[0] == '1':
                    parts[0] = '0'
                    new_lines.append(' '.join(parts))

            # If the file has valid tumor lines, save it
            if new_lines:
                with open(fpath, 'w') as file:
                    file.write('\n'.join(new_lines))
            else:
                os.remove(fpath)  # No tumors — delete label file


In [ ]:
# Install the official Ultralytics YOLOv8 package
!pip install -q ultralytics

# Check installation and version
import ultralytics
print("Ultralytics YOLO version:", ultralytics.__version__)

In [ ]:
from ultralytics import YOLO
import os
from google.colab import files
import matplotlib.pyplot as plt
import pandas as pd

# Best + Enhanced Hyperparameters
best_params = {
    'epochs': 200,               # Extended training
    'imgsz': 512,
    'batch': 16,
    'lr0': 0.002,
    'momentum': 0.937,
    'dropout': 0.2,
    'freeze': 0,                 # Unfreeze all layers for full fine-tuning
    'weight_decay': 0.0005
}

# Train with enhancements
exp_name = 'final_boosted_trial_v2'
model = YOLO('yolov8s.pt')  # pretrained YOLOv8s

model.train(
    data='/content/data/tumor_dataset.yaml',
    epochs=best_params['epochs'],
    imgsz=best_params['imgsz'],
    batch=best_params['batch'],
    lr0=best_params['lr0'],
    momentum=best_params['momentum'],
    weight_decay=best_params['weight_decay'],
    dropout=best_params['dropout'],
    freeze=best_params['freeze'],
    patience=25,                        # Longer patience for EarlyStopping
    optimizer='AdamW',
    cos_lr=True,                        # Cosine LR schedule
    mixup=0.1,                          # Mixup augmentation
    auto_augment='randaugment',        # RandAugment policy
    hsv_h=0.015,                        # Color jitter: hue
    hsv_s=0.7,                          # Saturation
    hsv_v=0.4,                          # Value (brightness)
    translate=0.1,                      # Shift
    scale=0.4,                          # Zoom
    shear=2.0,                          # Shear transform
    perspective=0.0005,                 # Perspective distortion
    flipud=0.1,                         # Vertical flip
    fliplr=0.5,                         # Horizontal flip
    mosaic=1.0,                         # Mosaic augmentation (default enabled, but reinforce)
    label_smoothing=0.01,              # Prevent overconfident predictions
    warmup_epochs=5,                   # Warmup to stabilize learning
    box=0.05,                           # Box loss gain
    cls=0.3,                            # Class loss gain
    dfl=1.5,                            # Distribution focal loss (default but reinforced)
    project='enhanced_training_runs',
    name=exp_name,
    verbose=True,
    seed=42,                            # Reproducibility
    device='cuda'                       # Use GPU
)

# Evaluate final model
best_model_path = f'enhanced_training_runs/{exp_name}/weights/best.pt'
model = YOLO(best_model_path)
metrics = model.val()

# Calculate F1 score manually
precision = metrics.box.mp
recall = metrics.box.mr
f1_score = 2 * (precision * recall) / (precision + recall + 1e-6)  # small epsilon to avoid division by zero

# Save evaluation metrics
results = {
    'Mean Precision': precision,
    'Mean Recall': recall,
    'F1 Score': f1_score,
    'mAP@0.5': metrics.box.map50,
    'mAP@0.5:0.95': metrics.box.map
}
df = pd.DataFrame([results])
#df.to_csv("final_metrics2.csv", index=False)
#print(df)
#files.download("final_metrics2.csv")

# Download best model
#files.download(best_model_path)

# Predict for manual inspection
model.predict(source='/content/data/images/val', save=True, conf=0.25)
print("Predictions saved to /runs/detect/predict")
